In [7]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils_imagenet
import logging
import argparse
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from model_search import Network
from architect import Architect


import genotypes

import copy



import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
#from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import re
#import albumentations as albu
#from albumentations.pytorch import ToTensor
#from catalyst.data import Augmentor
#import torchxrayvision as xrv


import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
#from catalyst.data import Augmentor
from skimage.io import imread, imsave
import skimage


parser = argparse.ArgumentParser("imagenet")
parser.add_argument('--data', type=str, default='/home/jiahzhao/Data/imagenet', help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=2, help='batch size')
parser.add_argument('--learning_rate', type=float, default=0.025, help='init learning rate')
parser.add_argument('--learning_rate_min', type=float, default=0.001, help='min learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=3e-4, help='weight decay')
parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--epochs', type=int, default=100, help='num of training epochs')
parser.add_argument('--init_channels', type=int, default=8, help='num of init channels')
parser.add_argument('--layers', type=int, default=8, help='total number of layers')
parser.add_argument('--model_path', type=str, default='saved_models', help='path to save the model')
parser.add_argument('--cutout', action='store_true', default=False, help='use cutout')
parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
parser.add_argument('--drop_path_prob', type=float, default=0.3, help='drop path probability')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--seed', type=int, default=100, help='random seed')
parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')
parser.add_argument('--train_portion', type=float, default=0.5, help='portion of training data')
parser.add_argument('--unrolled', action='store_true', default=True, help='use one-step unrolled validation loss')
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay', type=float, default=1e-3, help='weight decay for arch encoding')
args = parser.parse_args(args=[])



CIFAR_CLASSES = 1000

In [8]:
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)


criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
model = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model = model.cuda()
logging.info("param size = %fMB", utils_imagenet.count_parameters_in_MB(model))

In [9]:
optimizer = torch.optim.SGD(
      model.parameters(),
      args.learning_rate,
      momentum=args.momentum,
      weight_decay=args.weight_decay)

traindir = os.path.join(args.data, 'train')
validdir = os.path.join(args.data, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_data = dset.ImageFolder(
traindir,
transforms.Compose([
  transforms.RandomResizedCrop(224),
  transforms.RandomHorizontalFlip(),
  transforms.ColorJitter(
    brightness=0.4,
    contrast=0.4,
    saturation=0.4,
    hue=0.2),
  transforms.ToTensor(),
  normalize,
]))


num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(args.train_portion * num_train))
split = int(np.floor(0.001 * num_train))
split2 = int(np.floor(0.002 * num_train))



train_queue = torch.utils.data.DataLoader(
      train_data, batch_size=args.batch_size,
      sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
      pin_memory=True, num_workers=2)

valid_queue = torch.utils.data.DataLoader(
      train_data, batch_size=args.batch_size,
      sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:split2]),
      pin_memory=True, num_workers=2)


In [10]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])


# In[80]:


batchsize=10
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample




trainset = CovidCTDataset(root_dir='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Images-processed',
                          txt_COVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/COVID/trainCT_COVID.txt',
                          txt_NonCOVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                          transform= train_transformer)
valset = CovidCTDataset(root_dir='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Images-processed',
                          txt_COVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/COVID/valCT_COVID.txt',
                          txt_NonCOVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/NonCOVID/valCT_NonCOVID.txt',
                          transform= val_transformer)
testset = CovidCTDataset(root_dir='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Images-processed',
                          txt_COVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/COVID/testCT_COVID.txt',
                          txt_NonCOVID='/data/jiahzhao/COVID-CT-master_SEARCH_ON_cifar/Data-split/NonCOVID/testCT_NonCOVID.txt',
                          transform= val_transformer)
print(trainset.__len__())
print(valset.__len__())
print(testset.__len__())

batchsize=2

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=True, shuffle=True,pin_memory=True, num_workers=2)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=2)

##we use the validation performance on COVID dataset after transfering on training dataset.


424
116
200


In [11]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, float(args.epochs), eta_min=args.learning_rate_min)


alpha = None
device = 'cuda'

model_covid = Network(args.init_channels, 2, args.layers, criterion)
model_covid = model_covid.cuda()
model_covid.to(device)


def tranfer_weights(model,model_covid):
    model_covid = torch.nn.DataParallel(model_covid)   
    model = torch.nn.DataParallel(model)
    newmodel_dict=model_covid.state_dict()
    premodel_dict=model.state_dict()
    new_list=list(newmodel_dict.keys())
    pre_list=list(premodel_dict.keys())
    for i in range(4184):####The model contains 4186 keys, we need exclude the last layer
        newmodel_dict[new_list[i]]=premodel_dict[pre_list[i]]
    
    model_covid.load_state_dict(newmodel_dict)

    model_covid=model_covid.module
    return model_covid

model_covid=tranfer_weights(model,model_covid)


#dict_name=list(model.state_dict())
#for i,p in enumerate(dict_name):
    #print(i,p)

architect = Architect(model_covid, args)

In [12]:
optimizer_covid = optim.Adam(model_covid.parameters(), lr=0.0001)

def train_covid(train_loader, model_covid, criterion, optimizer):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    model_covid.train()
    
    loss = 0
    train_correct = 0

    for step, batch_samples in enumerate(train_loader):
        
        input, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
     
        target = target.cuda(async=True)
        input = input.cuda()
        input = Variable(input)
        target = Variable(target)

        optimizer.zero_grad()
        logits= model_covid(input)
        loss = criterion(logits, target)

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()

def get_weights_from_arch(arch_comb):
    k = sum(1 for i in range(model._steps) for n in range(2+i))
    num_ops = len(genotypes.PRIMITIVES)
    n_nodes = model._steps

    alphas_normal = Variable(torch.zeros(k, num_ops).cuda(), requires_grad=False)
    alphas_reduce = Variable(torch.zeros(k, num_ops).cuda(), requires_grad=False)

    offset = 0
    for i in range(n_nodes):
        normal1 = np.int_(arch_comb[0][2*i])
        normal2 = np.int_(arch_comb[0][2*i+1])
        reduce1 = np.int_(arch_comb[1][2*i])
        reduce2 = np.int_(arch_comb[1][2*i+1])
        alphas_normal[offset+normal1[1],normal1[0]] = 1
        alphas_normal[offset+normal2[1],normal2[0]] = 1
        alphas_reduce[offset+reduce1[1],reduce1[0]] = 1
        alphas_reduce[offset+reduce2[1],reduce2[0]] = 1
        offset += (i+2)

    model_weights = [
      alphas_normal,
      alphas_reduce,
    ]
    return model_weights


def set_model_weights(model, weights):
    model.alphas_normal = weights[0]
    model.alphas_reduce = weights[1]
    model._arch_parameters = [model.alphas_normal, model.alphas_reduce]

from genotypes import PRIMITIVES
def _parse_gene(weights):
    gene = []
    n = 2
    start = 0
    for i in range(4):
        end = start + n
        W = weights[start:end].copy()
        edges = sorted(range(i + 2), key=lambda x: -max(W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
        for j in edges:
            k_best = None
            for k in range(len(W[j])):
                if k != PRIMITIVES.index('none'):
                    if k_best is None or W[j][k] > W[j][k_best]:
                        k_best = k
            gene.append((k_best, j))
        start = end
        n += 1
    return gene


    

def train(train_queue, valid_queue, model,model_covid, architect, criterion, optimizer, lr):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    top5 = utils_imagenet.AvgrageMeter()

    for step, (input, target) in enumerate(train_queue):
        model.train()
        n = input.size(0)

        input = Variable(input, requires_grad=False).cuda()
        target = Variable(target, requires_grad=False).cuda(async=True)

        # get a random minibatch from the search queue with replacement
        batch_next = next(iter(val_loader))
        input_search, target_search = batch_next['img'].to(device), batch_next['label'].to(device)

        input_search = Variable(input_search, requires_grad=False).cuda()
        target_search = Variable(target_search, requires_grad=False).cuda(async=True)
        
        train_covid(train_loader, model_covid, criterion, optimizer_covid)######train on covid

        architect.step(input, target, input_search, target_search, lr, optimizer, unrolled=False)

        arch_param_save=model_covid.arch_parameters()
    
        #temp= opt.initial_temp * np.exp(-opt.anneal_rate * step)
        temp= 2.5* np.exp(-0.00003  * step)
        temperature=torch.tensor([temp]).type(torch.FloatTensor)
        
        alpha_nor=torch.tensor(arch_param_save[0]).type(torch.FloatTensor)
        
        while True:
            gumbels = -torch.empty_like(alpha_nor).exponential_().log()
            logits  = (alpha_nor.log_softmax(dim=1) + gumbels) / temp
            probs   = nn.functional.softmax(logits, dim=1)
            index   = probs.max(-1, keepdim=True)[1]
            one_h   = torch.zeros_like(logits).scatter_(-1, index, 1.0)
            hardwts = one_h - probs.detach() + probs
            if (torch.isinf(gumbels).any()) or (torch.isinf(probs).any()) or (torch.isnan(probs).any()):
                continue
            else: break
        gene_normal = _parse_gene(F.softmax(probs, dim=-1).data.cpu().numpy())    
        
        alpha_red=torch.tensor(arch_param_save[1]).type(torch.FloatTensor)        
        while True:
            gumbels = -torch.empty_like(alpha_red).exponential_().log()
            logits  = (alpha_nor.log_softmax(dim=1) + gumbels) / temp
            probs   = nn.functional.softmax(logits, dim=1)
            index   = probs.max(-1, keepdim=True)[1]
            one_h   = torch.zeros_like(logits).scatter_(-1, index, 1.0)
            hardwts = one_h - probs.detach() + probs
            if (torch.isinf(gumbels).any()) or (torch.isinf(probs).any()) or (torch.isnan(probs).any()):
                continue
            else: break        
        
        gene_reduce = _parse_gene(F.softmax(probs, dim=-1).data.cpu().numpy())
        
        arch_gene = [gene_normal,gene_reduce]
        model_weights=get_weights_from_arch(arch_gene)        
        set_model_weights(model,model_weights)      
        logits = model(input)        
        loss_cur=criterion(logits, target) 


        loss=loss_cur
        
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()
        
        objs.update(loss.data, n)

        
        set_model_weights(model,arch_param_save)###########################set back


    return objs.avg

In [13]:
for epoch in range(50):

    scheduler.step()
    lr = scheduler.get_lr()[0]
    logging.info('epoch %d lr %e', epoch, lr)

    genotype = model.genotype()
    logging.info('genotype = %s', genotype)

    print(F.softmax(model.alphas_normal, dim=-1))
    print(F.softmax(model.alphas_reduce, dim=-1))

    # training
    train_obj = train(train_queue, val_loader, model,model_covid, architect, criterion, optimizer, lr)
    
    
    targetlist, scorelist, predlist=infer(val_loader, model, criterion,epoch)
    
    print(model.genotype())

tensor([[0.1248, 0.1248, 0.1250, 0.1251, 0.1250, 0.1250, 0.1251, 0.1253],
        [0.1251, 0.1249, 0.1250, 0.1249, 0.1248, 0.1251, 0.1251, 0.1251],
        [0.1249, 0.1251, 0.1251, 0.1248, 0.1250, 0.1249, 0.1251, 0.1251],
        [0.1252, 0.1250, 0.1251, 0.1250, 0.1248, 0.1249, 0.1248, 0.1252],
        [0.1249, 0.1252, 0.1249, 0.1251, 0.1250, 0.1249, 0.1250, 0.1250],
        [0.1248, 0.1250, 0.1249, 0.1252, 0.1250, 0.1252, 0.1249, 0.1250],
        [0.1250, 0.1250, 0.1249, 0.1248, 0.1250, 0.1251, 0.1251, 0.1250],
        [0.1249, 0.1250, 0.1251, 0.1250, 0.1251, 0.1250, 0.1248, 0.1252],
        [0.1251, 0.1248, 0.1248, 0.1251, 0.1250, 0.1249, 0.1251, 0.1251],
        [0.1250, 0.1250, 0.1249, 0.1251, 0.1249, 0.1249, 0.1250, 0.1252],
        [0.1250, 0.1248, 0.1249, 0.1250, 0.1250, 0.1250, 0.1251, 0.1251],
        [0.1251, 0.1249, 0.1251, 0.1249, 0.1250, 0.1249, 0.1251, 0.1249],
        [0.1252, 0.1250, 0.1249, 0.1251, 0.1251, 0.1248, 0.1249, 0.1250],
        [0.1249, 0.1250, 0.1250, 0.125

/data/jiahzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/data/jiahzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/data/jiahzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:126: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/data/jiahzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:151: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
Traceback (most recent call last):
  File "/data/jiahzhao/anaconda3/lib/python3

KeyboardInterrupt: 